In [ ]:
# Function: Create generator capacity dataset whose format is suitable for regression
# Date: 2021-7-25
# Output: capacity3.xlsx
# Writer Mingyi Li

In [1]:
import numpy as np
import pandas as pd


In [2]:
data = pd.read_excel(r'Z:/User/Documents/generator-distribution.xls')

In [3]:
data.head()

,FID,Join_Count,TARGET_FID,JOIN_FID,chname,ename,gbcode,code,province,unit,name_cn,sponsor,parent,capacity,year,location,name_gener,lat,lon
0,0,0,0,-1,涡阳县,Guoyang,341621,0,,,,,,0,0,,,0.000000,0.00000
1,1,0,1,-1,鼓楼区,Gulou,350102,0,,,,,,0,0,,,0.000000,0.00000
2,2,0,2,-1,固镇县,Guzhen,340323,0,,,,,,0,0,,,0.000000,0.00000
3,3,0,3,-1,古田县,Gutian,350922,0,,,,,,0,0,,,0.000000,0.00000
4,4,1,4,123,贵池区,Guichi,341702,0,Anhui,chizhou jiuhua power station unit 1,神华神皖池州九华发电公司,China Shenhua Energy Co Ltd,Shenhua Group,300,2005,Jiangkou,chizhou jiuhua power station,30.753111,117.59066


In [4]:
data.columns

Index(['FID', 'Join_Count', 'TARGET_FID', 'JOIN_FID', 'chname', 'ename',
       'gbcode', 'code', 'province', 'unit', 'name_cn', 'sponsor', 'parent',
       'capacity', 'year', 'location', 'name_gener', 'lat', 'lon'],
      dtype='object')

In [5]:
dataset = data.reindex(['TARGET_FID','chname','gbcode','province','capacity','year'], axis=1)

In [6]:
dataset = dataset.reset_index()

In [7]:
# capacity = dataset.sort_values(by=['TARGET_FID','year']).groupby('TARGET_FID')['capacity'].cumsum()
capacity = dataset.sort_values(by=['gbcode','year']).groupby('gbcode')['capacity'].cumsum()

In [8]:
dataset['total_capacity'] = capacity

In [71]:
# dataset = dataset.rename({'TARGET_FID':'FID'},axis=1)

In [9]:
max_capacity = dataset.groupby(['gbcode','year'])['total_capacity'].max()

In [10]:
max_capacity = max_capacity.reset_index()

In [11]:
max_capacity

,gbcode,year,total_capacity
0,110101,0,0
1,110102,0,0
2,110103,0,0
3,110104,0,0
4,110105,0,0
...,...,...,...
3638,659003,0,0
3639,659004,2011,2160
3640,659004,2012,2880
3641,659004,2014,5080


In [12]:
dataset2 = dataset.merge(max_capacity ,left_on=['gbcode','year'], right_on=['gbcode','year'])

In [13]:
dataset2['year'].unique()

array([   0, 2005, 1993, 2014, 2007, 2013, 2008, 2011, 2006, 2009, 2015,
       1998, 1999, 2000, 2001, 1991, 1992, 1996, 1985, 1986, 1997, 2010,
       2004, 2012, 1995, 1988, 1994, 1989, 1990, 2002, 2003, 2016, 1987,
       1975, 1970, 1972, 1973, 1974, 1983, 1978, 1984, 1982, 1979, 1980],
      dtype=int64)

In [14]:
dataset2.head()

,index,TARGET_FID,chname,gbcode,province,capacity,year,total_capacity_x,total_capacity_y
0,0,0,涡阳县,341621,,0,0,0,0
1,1,1,鼓楼区,350102,,0,0,0,0
2,2,2,固镇县,340323,,0,0,0,0
3,3,3,古田县,350922,,0,0,0,0
4,4,4,贵池区,341702,Anhui,300,2005,300,600


In [15]:
counties = dataset2['gbcode'].unique()

In [16]:
dataset2

,index,TARGET_FID,chname,gbcode,province,capacity,year,total_capacity_x,total_capacity_y
0,0,0,涡阳县,341621,,0,0,0,0
1,1,1,鼓楼区,350102,,0,0,0,0
2,2,2,固镇县,340323,,0,0,0,0
3,3,3,古田县,350922,,0,0,0,0
4,4,4,贵池区,341702,Anhui,300,2005,300,600
...,...,...,...,...,...,...,...,...,...
4877,4837,3117,滨海新区,120116,Tianjin,350,2014,2700,2700
4878,4841,3121,北辰区,120113,,0,0,0,0
4879,4848,3128,武清区,120114,,0,0,0,0
4880,4865,3139,蓟县,120225,,0,0,0,0


In [17]:
dataset3 = dataset2.groupby(['gbcode','year']).max()

In [18]:
dataset3 = dataset3.reset_index()

In [19]:
demo = dataset3[dataset3['chname'] =='滨海新区']

In [20]:
demo

,gbcode,year,index,TARGET_FID,chname,province,capacity,total_capacity_x,total_capacity_y
40,120116,2009,4835,3117,滨海新区,Tianjin,1000,2000,2000
41,120116,2014,4837,3117,滨海新区,Tianjin,350,2700,2700


In [21]:
DEMO_YEAR = pd.DataFrame(np.arange(1970,2017), columns=['year'])

In [22]:
def fill_method(df):
    # fill the whole time range with capacity including 0
    tmp = df.merge(DEMO_YEAR, how = 'outer').sort_values(by='year').fillna(method='ffill')
    # excluding column total_capacity_y, filling the rest
    df = tmp.iloc[:,:-1].fillna(method='bfill')
    df['total_capacity_y'] = tmp['total_capacity_y']
    return df.fillna(0)

In [23]:
dataset4 = dataset3.groupby('gbcode').apply(fill_method)

In [24]:
dataset4

gbcode  year   index  TARGET_FID chname  province  capacity  \
gbcode                                                                     
110101 0   110101.0     0   320.0       211.0    东城区                 0.0   
       1   110101.0  1970   320.0       211.0    东城区                 0.0   
       2   110101.0  1971   320.0       211.0    东城区                 0.0   
       3   110101.0  1972   320.0       211.0    东城区                 0.0   
       4   110101.0  1973   320.0       211.0    东城区                 0.0   
...             ...   ...     ...         ...    ...       ...       ...   
659004 1   659004.0  2012  4871.0      3142.0   五家渠市  Xinjiang     360.0   
       45  659004.0  2013  4871.0      3142.0   五家渠市  Xinjiang     360.0   
       2   659004.0  2014  4872.0      3142.0   五家渠市  Xinjiang    1100.0   
       3   659004.0  2015  4873.0      3142.0   五家渠市  Xinjiang    1100.0   
       46  659004.0  2016  4873.0      3142.0   五家渠市  Xinjiang    1100.0   

           total_capacity_x  total_capacity_y  
gbcode                                         
110101 0                0.0               0.0  
       1                0.0               0.0  
       2                0.0               0.0  
       3                0.0               0.0  
       4                0.0               0.0  
...                     ...               ...  
659004 1             2880.0            2880.0  
       45            2880.0            2880.0  
       2             5080.0            5080.0  
       3             7280.0            7280.0  
       46            7280.0            7280.0  

[137401 rows x 9 columns]

In [25]:
dataset4 = dataset4[dataset4['year'] != 0]

In [38]:
dataset5 = dataset4.reindex(['gbcode','year','total_capacity_y','province'],axis=1)

In [39]:
dataset5 = dataset5.rename({'total_capacity_y':'total_capacity'},axis=1)

In [40]:
dataset5 = dataset5[dataset5['year'] > 1990]

In [41]:
dataset5.to_excel('capacity3.xlsx',index=False)